In [1]:
#r "nuget:Microsoft.DotNet.Interactive.SqlServer, *-*"

Installed Packages Microsoft.DotNet.Interactive.SqlServer, 1.0.0-beta.25323.1

Loading extension script from `C:\Users\Pedro_Carvalho1\.nuget\packages\microsoft.dotnet.interactive.sqlserver\1.0.0-beta.25323.1\interactive-extensions\dotnet\extension.dib`

Query Microsoft SQL Server databases. 
 This extension adds support for connecting to Microsoft SQL Server databases using the #!connect mssql magic command. For more information, run a cell using the #!sql magic command.

In [2]:
using System.IO;

var connStr = File.ReadAllText(@"C:\Users\Pedro_Carvalho1\conn_str.txt");

#!share --from csharp connStr

In [3]:
#!connect mssql --kernel-name sql --connection-string @connStr

Kernel added: #!sql-sql

In [4]:
#!sql-sql
-- Garantir que o schema existe
if not exists (select * from sys.schemas where name = 'sql008')
    exec('create schema sql008');
go

-- Se a tabela já existir, elimina antes
if object_id('sql008.Activity', 'U') is not null
    drop table sql008.Activity;
go

-- Criação da tabela
CREATE TABLE sql008.Activity (
    player_id    INT NOT NULL,
    device_id    INT NOT NULL,
    event_date   DATE NOT NULL,
    games_played INT NOT NULL,
    CONSTRAINT PK_Activity PRIMARY KEY (player_id, event_date)
);
GO

INSERT INTO sql008.Activity (player_id, device_id, event_date, games_played)
VALUES
(1, 2, '2016-03-01', 5),
(1, 2, '2016-03-02', 6),
(2, 3, '2017-06-25', 1),
(3, 1, '2016-03-02', 0),
(3, 4, '2018-07-03', 5);
GO


Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

(5 rows affected)

In [5]:
#!sql-sql
select * from sql008.Activity

(5 rows affected)

player_id,device_id,event_date,games_played
1,2,2016-03-01 00:00:00Z,5
1,2,2016-03-02 00:00:00Z,6
2,3,2017-06-25 00:00:00Z,1
3,1,2016-03-02 00:00:00Z,0
3,4,2018-07-03 00:00:00Z,5


In [8]:
#!sql-sql

WITH ordered_logins AS (
    SELECT 
        player_id,
        event_date,
        ROW_NUMBER() OVER (PARTITION BY player_id ORDER BY event_date ASC) AS rn
    FROM sql008.Activity
),
first_login AS (
    SELECT player_id, event_date AS first_date
    FROM ordered_logins
    WHERE rn = 1
),
next_day_login AS (
    SELECT DISTINCT f.player_id
    FROM first_login f
    JOIN sql008.Activity a
        ON a.player_id = f.player_id
       AND a.event_date = DATEADD(DAY, 1, f.first_date)
)
SELECT 
    CAST(ROUND(
        1.0 * 
        (SELECT COUNT(*) FROM next_day_login) /
        (SELECT COUNT(DISTINCT player_id) FROM sql008.Activity),
    2) AS DECIMAL(4,2)) AS fraction;


(1 row affected)

fraction
0.33
